<a href="https://colab.research.google.com/github/andrnsgnmez/Red-convolucional-miniproyecto-3/blob/main/MINI_PROYECTO_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mini proyecto 3


In [ ]:
h